In [19]:
### imports ###
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy as sc

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import torch.optim as optim


import pickle


# add paths to access shared code
import sys
sys.path.append("..")
sys.path.append("../scripts/")

# import library implementing models
import neuroprob as nprb
from neuroprob import utils

# import utility code for model building/training/loading
import lib
import HDC

# get GPU device if available
gpu_dev = 1
dev = utils.pytorch.get_device(gpu=gpu_dev)

# use custom plotting settings
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
PyTorch version: 1.12.1+cu102
Using device: cuda:1


In [10]:
 mice_sessions = {
    'Mouse12': ['120806']#, '120807', '120809', '120810' ], # '120808' is missing position files
#    'Mouse17': ['130125', '130128', '130129', '130130', '130131', '130201', '130202', '130203', '130204'],
#    'Mouse20': ['130514', '130515', '130516', '130517'], # '130520' hdc models fails
#    'Mouse24': ['131213', '131216', '131217','131218'],
#    'Mouse25': ['140123', '140124', '140128', '140129', '140130', '140131', '140203', '140204', '140205', '140206'],
#    'Mouse28': ['140310', '140311', '140312', '140313', '140317', '140318'],
} 

In [16]:
for mouse_id in mice_sessions.keys():
    for session_id in mice_sessions[mouse_id]:
        for subset in ['hdc']:#, 'nonhdc']:
            try:
                checkpoint_dir = '/scratches/ramanujan_2/vn283/HDC_PartIII/checkpoint/'

                phase = 'wake'

                bin_size = 160  # ms

                # data
                single_spikes = False
                dataset_dict = HDC.get_dataset(mouse_id, session_id, phase, subset, bin_size, 
                                               single_spikes, path='/scratches/ramanujan_2/dl543/HDC_PartIII/')

                model_dict = {
                    'seed': 123, 
                    'll_mode': 'U-ec-3',  # stands for universal count model with exponential-quadratic expansion and C = 3
                    'filt_mode': '',  # GLM couplings
                    'map_mode': 'svgp-64',  # a sparse variational GP mapping with 64 inducing points
                    'x_mode': 'hd-omega-speed-x-y-time',  # observed covariates (behaviour)
                    'z_mode': '',  # latent covariates
                    'hist_len': 0, 
                    'folds': 5, 
                    'delays': [0], 
                    'neurons': dataset_dict['neurons'], 
                    'max_count': dataset_dict['max_count'], 
                    'bin_size': dataset_dict['bin_size'], 
                    'tbin': dataset_dict['tbin'], 
                    'model_name': dataset_dict['name'], 
                    'tensor_type': torch.float, 
                    'jitter': 1e-5, 
                }
                
            except FileNotFoundError:
                continue

units:  85  hdc units:  22


In [20]:
checkpoint_dir = '/scratches/ramanujan_2/vn283/HDC_PartIII/checkpoint/'
cv_run = -1  # test set is last 1/5 of dataset time series
delay = 0
batch_size = 5000  # size of time segments of each batch in dataset below

# model and train/test split based on which validation fold is given by cv_run
modelfit, training_results, fit_set, validation_set = lib.models.load_model(
    checkpoint_dir, model_dict, dataset_dict, HDC.enc_used, 
    delay, cv_run, batch_size, gpu_dev
)
plt.plot(training_results['training_loss'])
plt.title(mouse_id+'-'+session_id+'-'+subset)
plt.show()

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.